In [1]:
import pandas as pd
import os
import random
from tqdm import tqdm
import pickle
from sklearn.preprocessing import MinMaxScaler

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
#TODO when i am filling values from current index back to a length of window size, i didn't consider the previous interaction. this has to be updated.

In [2]:
regenerate_context_data = True
sequence_augmentation = True
whole_session_context = False
model_test_run = False
data_autoencoder = False
pad_to_window_size = True
feat_engg = True

PATH_TO_LOAD = '../data/04_Merged'
if feat_engg:
    combined_context_path = '../data/06_context_feat_engg/data_featue_engineering.csv'
    augmentation_folder = 'featengg/' if sequence_augmentation else 'non_aug/'
else:
    combined_context_path = '../data/05_Interaction_Sequences/context.csv'
    augmentation_folder = 'fix/' if sequence_augmentation else 'non_aug/'

window = 100 #seconds

base_path = '../datasets/sequential/'
augmentation_folder = 'aug/' if sequence_augmentation else 'non_aug/'
if model_test_run:
    augmentation_folder = 'test/aug/' if sequence_augmentation else 'test/non_aug/'

sequence_context_path = f'{base_path}{augmentation_folder}parameters/sequence_context.csv'
parameter_path = f'{base_path}{augmentation_folder}parameters'
train_session_path = f'{base_path}{augmentation_folder}parameters/train_sessions.pkl'
test_session_path = f'{base_path}{augmentation_folder}parameters/test_sessions.pkl'
train_dynamic_context_path = f'{base_path}{augmentation_folder}dynamic_context/train.csv'
test_dynamic_context_path = f'{base_path}{augmentation_folder}dynamic_context/test.csv'
train_static_context_path = f'{base_path}{augmentation_folder}static_context/train.csv'
test_static_context_path = f'{base_path}{augmentation_folder}static_context/test.csv'

In [3]:
all_columns = ['index', 'avg_irradiation', 'steering_speed', 'temperature_out', 'hour',
       'month', 'odometer', 'light_sensor_rear', 'light_sensor_front',
       'temperature_in', 'KBI_speed', 'soc', 'ESP_speed', 'latitude',
       'longitude', 'seatbelt_codriver', 'seatbelt_rear_l', 'seatbelt_rear_m',
       'seatbelt_rear_r', 'CHA_ESP_drive_mode', 'CHA_MO_drive_mode',
       'rain_sensor', 'street_category', 'kickdown', 'altitude',
       'driving_program', 'datetime', 'session', 'Label', 'ID',
       'FunctionValue', 'domain', 'BeginTime', 'time_second',
       'distance_driven', 'ts_normalized', 'weekday']

selected = [ 'avg_irradiation', 'steering_speed', 'temperature_out', 'hour',
       'month', 'light_sensor_rear', 'light_sensor_front',
       'temperature_in', 'KBI_speed', 'soc', 'latitude',
       'longitude', 'seatbelt_codriver', 'seatbelt_rear_l',
       'seatbelt_rear_r', 'street_category', 'altitude',
       'datetime', 'session', 'time_second',
       'distance_driven', 'weekday'
]

bad_quality = ['CHA_ESP_drive_mode', 
             'CHA_MO_drive_mode',
             'rain_sensor',
             'kickdown',
             'ESP_speed',
             'seatbelt_rear_m',
            'driving_program',
            'ts_normalized'
             ]

dynamic_context_var = ['avg_irradiation', 'steering_speed', 'temperature_out', 
                       'light_sensor_rear', 'light_sensor_front', 
                       'temperature_in', 'KBI_speed', 'soc', 'latitude',
                       'longitude',  'street_category', 'altitude','time_second',
                       'distance_driven']
static_context_var = ['car_id', 'month', 'weekday', 'hour', 'seatbelt_codriver', 'seatbelt_rear_l',
                       'seatbelt_rear_r',]

status_static_context_var = ['ess_status', 'current_drive_mode', 'current_clima_mode', 'current_media_source', # status static context
                     'nav_guidance_status', 'proximity_to_parking_spot', 'phone_status',
                     'bluetooth_connected', 'phone_os',
                     'new_bluetooth_device_to_pair']
#Todo take average of these value over a window
constant_context_var = ['avg_irradiation','temperature_out'] #to be filled

In [5]:
def load_context(vehicle):
    context_lists = dynamic_context_var + static_context_var + ['session', 'datetime', 'Label']
    context_lists.remove('car_id')
    df = pd.read_csv(os.path.join(PATH_TO_LOAD, vehicle + "_merged.csv"), parse_dates=['datetime'], usecols=context_lists, low_memory=False)
    df = df.sort_values(by=['session','datetime'])
    return df

vehicles = ['SEB880','SEB882','SEB883','SEB885','SEB888','SEB889']
context_data_list = []
for vehicle in tqdm(vehicles):
    context_curr = load_context(vehicle)
    context_curr['car_id'] = vehicle
    context_data_list.append(context_curr)
context_data = pd.concat(context_data_list, axis=0).reset_index()
# context_data.to_csv(combined_context_path, index=False)

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:55<00:00,  9.29s/it]


In [6]:
context_data

,index,avg_irradiation,steering_speed,temperature_out,hour,month,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,seatbelt_rear_r,street_category,altitude,datetime,session,Label,time_second,distance_driven,weekday,car_id
0,0,0.000000,0.000000,0.0,20,9,0.0,0.000000,0.0,0.00,...,0.0,0.0,0.000000,2022-09-07 20:08:33,7.0,NaN,0.0,0.0,2,SEB880
1,1,0.000000,0.000000,0.0,20,9,0.0,0.000000,0.0,0.00,...,0.0,7.0,0.000000,2022-09-07 20:08:34,7.0,NaN,1.0,10845.0,2,SEB880
2,2,0.000000,0.000000,25.0,20,9,0.0,0.000000,0.0,0.00,...,0.0,7.0,0.000000,2022-09-07 20:08:35,7.0,NaN,2.0,10845.0,2,SEB880
3,3,0.000000,0.000000,25.0,20,9,0.0,0.000000,27.0,0.00,...,0.0,7.0,0.000000,2022-09-07 20:08:36,7.0,NaN,3.0,10845.0,2,SEB880
4,4,0.000000,0.000000,25.0,20,9,0.0,0.000000,27.0,0.00,...,0.0,7.0,0.000000,2022-09-07 20:08:36,7.0,NaN,3.0,10845.0,2,SEB880
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10124879,1360504,53.755324,0.000000,13.5,17,3,17600.0,6126.000000,28.5,92.48,...,1.0,4.0,474.000000,2023-03-31 17:09:13,5399.0,NaN,5360.0,146.0,4,SEB889
10124880,1360505,50.000000,0.000000,13.5,17,3,17200.0,5920.781157,28.5,93.12,...,1.0,7.0,474.000000,2023-03-31 17:09:14,5399.0,NaN,5361.0,146.0,4,SEB889
10124881,1360506,56.226313,15.000000,13.5,17,3,18000.0,5699.908827,28.5,93.44,...,1.0,4.0,474.000000,2023-03-31 17:09:15,5399.0,NaN,5362.0,146.0,4,SEB889
10124882,1360507,60.000000,0.000000,13.5,17,3,17600.0,5437.218229,28.5,93.76,...,1.0,4.0,472.468149,2023-03-31 17:09:16,5399.0,NaN,5363.0,146.0,4,SEB889


In [7]:
# context_data = pd.read_csv(combined_context_path, parse_dates=['datetime'], low_memory=False)
# vehicle_list = context_data.car_id.unique().tolist()
# random.shuffle(vehicle_list)
# vehicle_dict = {}
# used_values = set()
# for vehicle in tqdm(vehicle_list):
#     value = random.randint(1, len(vehicle_list))
#     while value in used_values:
#         value = random.randint(1, len(vehicle_list))
#     vehicle_dict[vehicle] = value
#     used_values.add(value)

# context_data['car_id'] = context_data['car_id'].map(vehicle_dict)
context_data['car_id'], _ = pd.factorize(context_data['car_id'])
context_data = context_data.sort_values(by=['session','datetime'])

In [8]:
context_data

,index,avg_irradiation,steering_speed,temperature_out,hour,month,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,seatbelt_rear_r,street_category,altitude,datetime,session,Label,time_second,distance_driven,weekday,car_id
0,0,0.000000,0.000000,0.0,20,9,0.0,0.000000,0.0,0.00,...,0.0,0.0,0.000000,2022-09-07 20:08:33,7.0,NaN,0.0,0.0,2,0
1,1,0.000000,0.000000,0.0,20,9,0.0,0.000000,0.0,0.00,...,0.0,7.0,0.000000,2022-09-07 20:08:34,7.0,NaN,1.0,10845.0,2,0
2,2,0.000000,0.000000,25.0,20,9,0.0,0.000000,0.0,0.00,...,0.0,7.0,0.000000,2022-09-07 20:08:35,7.0,NaN,2.0,10845.0,2,0
3,3,0.000000,0.000000,25.0,20,9,0.0,0.000000,27.0,0.00,...,0.0,7.0,0.000000,2022-09-07 20:08:36,7.0,NaN,3.0,10845.0,2,0
4,4,0.000000,0.000000,25.0,20,9,0.0,0.000000,27.0,0.00,...,0.0,7.0,0.000000,2022-09-07 20:08:36,7.0,NaN,3.0,10845.0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10124879,1360504,53.755324,0.000000,13.5,17,3,17600.0,6126.000000,28.5,92.48,...,1.0,4.0,474.000000,2023-03-31 17:09:13,5399.0,NaN,5360.0,146.0,4,5
10124880,1360505,50.000000,0.000000,13.5,17,3,17200.0,5920.781157,28.5,93.12,...,1.0,7.0,474.000000,2023-03-31 17:09:14,5399.0,NaN,5361.0,146.0,4,5
10124881,1360506,56.226313,15.000000,13.5,17,3,18000.0,5699.908827,28.5,93.44,...,1.0,4.0,474.000000,2023-03-31 17:09:15,5399.0,NaN,5362.0,146.0,4,5
10124882,1360507,60.000000,0.000000,13.5,17,3,17600.0,5437.218229,28.5,93.76,...,1.0,4.0,472.468149,2023-03-31 17:09:16,5399.0,NaN,5363.0,146.0,4,5


In [9]:
grouped_columns = ['session', 'datetime']
columns_to_average = dynamic_context_var + static_context_var
grouped_data = context_data.groupby(grouped_columns)[columns_to_average].mean().reset_index()
merged_data = pd.merge(context_data[['session', 'datetime', 'Label']], grouped_data, on=grouped_columns, suffixes=('', '_mean'))

# context_data['session'] = context_data['session'].astype(int)
# context_data = context_data[dynamic_context_var + static_context_var + ['session', 'datetime', 'Label']]
# context_data = context_data.groupby(['session', 'datetime'])[dynamic_context_var + static_context_var].mean().reset_index()

In [10]:
merged_data

,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,altitude,time_second,distance_driven,car_id,month,weekday,hour,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r
0,7.0,2022-09-07 20:08:33,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,...,0.000000,0.0,0.0,0.0,9.0,2.0,20.0,1.0,1.0,0.0
1,7.0,2022-09-07 20:08:34,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,...,0.000000,1.0,10845.0,0.0,9.0,2.0,20.0,1.0,1.0,0.0
2,7.0,2022-09-07 20:08:35,NaN,0.000000,0.000000,25.0,0.0,0.000000,0.0,0.00,...,0.000000,2.0,10845.0,0.0,9.0,2.0,20.0,1.0,1.0,0.0
3,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,27.0,0.00,...,0.000000,3.0,10845.0,0.0,9.0,2.0,20.0,1.0,1.0,0.0
4,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,27.0,0.00,...,0.000000,3.0,10845.0,0.0,9.0,2.0,20.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10124879,5399.0,2023-03-31 17:09:13,NaN,53.755324,0.000000,13.5,17600.0,6126.000000,28.5,92.48,...,474.000000,5360.0,146.0,5.0,3.0,4.0,17.0,1.0,1.0,1.0
10124880,5399.0,2023-03-31 17:09:14,NaN,50.000000,0.000000,13.5,17200.0,5920.781157,28.5,93.12,...,474.000000,5361.0,146.0,5.0,3.0,4.0,17.0,1.0,1.0,1.0
10124881,5399.0,2023-03-31 17:09:15,NaN,56.226313,15.000000,13.5,18000.0,5699.908827,28.5,93.44,...,474.000000,5362.0,146.0,5.0,3.0,4.0,17.0,1.0,1.0,1.0
10124882,5399.0,2023-03-31 17:09:16,NaN,60.000000,0.000000,13.5,17600.0,5437.218229,28.5,93.76,...,472.468149,5363.0,146.0,5.0,3.0,4.0,17.0,1.0,1.0,1.0


In [11]:
merged_data['Label'] = merged_data['Label'].replace('car/driveMode/0', 'car/driveMode/0.0')
merged_data['Label'] = merged_data['Label'].replace('car/driveMode/2', 'car/driveMode/2.0')
merged_data['Label'] = merged_data['Label'].replace('car/driveMode/3', 'car/driveMode/3.0')

merged_data['Label'] = merged_data['Label'].replace('car/charismaLevel/Abgesenkt', 'car/charismaLevel/change')
merged_data['Label'] = merged_data['Label'].replace('car/charismaLevel/Lift', 'car/charismaLevel/change')
merged_data['Label'] = merged_data['Label'].replace('car/charismaLevel/Mittel', 'car/charismaLevel/change')
merged_data['Label'] = merged_data['Label'].replace('car/charismaLevel/Tief', 'car/charismaLevel/change')
context_data = merged_data

In [12]:
cleaned_list = [x for x in context_data.Label.unique().tolist() if not pd.isna(x)]
sorted(cleaned_list)

['car/ESS/on',
 'car/Start/ParkAssistant',
 'car/charismaLevel/change',
 'car/driveMode/0.0',
 'car/driveMode/1.0',
 'car/driveMode/2.0',
 'car/driveMode/3.0',
 'clima/AC/ECO',
 'clima/AC/off',
 'clima/AC/on',
 'media/selectedSource/Bluetooth',
 'media/selectedSource/CarPlay',
 'media/selectedSource/Favorite',
 'media/selectedSource/Radio',
 'navi/Start/Address',
 'navi/Start/Favorite',
 'phone/Call/Favorite',
 'phone/Call/PersonX',
 'phone/Connect/NewDevice',
 'phone/Start/AndroidAuto',
 'phone/Start/CarPlay',
 'phone/goTo/Favorite']

In [13]:

context_data.to_csv(combined_context_path, index=False)

In [14]:
# TODO ess status - Untill the point ess is swtiched on, add a status that ess is off now - 1 on/ 0 off
# current drive mode - 0, 1, 2, 3
# current AC mode - Eco, on, off
# current selected source - bluetooth, carplay, favourite, radio, no source
# navigation guidance activation status - 0 - inactive/ 1 - active
# phone call status - 
# os of the phone connected
# new bluetooth device around status - 1 - yes and 0- no
# phone connected or not -  (needed to make a phone call)
# time since last interaction as a static variable
# convert street category into text highway, countryside etc
# season
# ventilation running
# racetrack proximity
# desired temperature - this can go in user embedding may ?
# user embedding is introduced as kind of memory
# current car temperature

In [15]:
context_data[context_data['Label']=='car/ESS/on']

,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,altitude,time_second,distance_driven,car_id,month,weekday,hour,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r
146831,205.0,2022-12-29 20:23:44,car/ESS/on,0.000000,0.000000,12.0,0.000000,0.000000,21.500000,22.037324,...,194.000000,870.0,14728.0,1.0,12.0,3.0,20.0,1.0,1.0,1.0
147046,205.0,2022-12-29 20:27:19,car/ESS/on,0.000000,0.000000,12.0,0.000000,0.000000,22.000000,0.000000,...,198.000000,1085.0,14729.0,1.0,12.0,3.0,20.0,1.0,1.0,1.0
3744925,1659.0,2022-12-22 10:31:00,car/ESS/on,6.419186,0.000000,9.5,4800.000000,1350.000000,26.000000,0.000000,...,474.000000,9614.0,28353.0,2.0,12.0,3.0,10.0,0.0,0.0,0.0
4334549,1908.0,2023-02-07 09:05:58,car/ESS/on,0.000000,0.000000,-3.0,0.000000,72.000000,22.500000,0.000000,...,338.000000,7219.0,34029.0,2.0,2.0,1.0,9.0,1.0,0.0,0.0
4574089,1997.0,2023-02-28 13:49:30,car/ESS/on,0.266636,0.000000,0.0,14398.797565,6041.855708,23.181415,0.000000,...,468.000000,22299.0,36521.0,2.0,2.0,1.0,13.0,0.0,0.0,0.0
5540311,2587.0,2022-10-21 10:56:37,car/ESS/on,60.000000,5.029246,14.5,17735.744925,5208.488347,28.000000,51.200000,...,694.000000,18495.0,10472.0,5.0,10.0,4.0,10.0,1.0,0.0,0.0
5540322,2587.0,2022-10-21 10:56:48,car/ESS/on,50.000000,0.000000,14.5,18000.000000,5890.053078,28.000000,58.240000,...,688.000000,18506.0,10472.0,5.0,10.0,4.0,10.0,1.0,0.0,0.0
5543739,2587.0,2022-10-21 11:53:44,car/ESS/on,70.000000,0.000000,13.0,22000.000000,4650.000000,28.000000,83.840000,...,931.706949,21922.0,10565.0,5.0,10.0,4.0,11.0,1.0,0.0,0.0
9930082,5256.0,2023-02-26 17:30:43,car/ESS/on,10.000000,0.000000,2.0,5200.000000,2153.365494,6.500000,67.203988,...,263.814125,270.0,4.0,6.0,2.0,6.0,17.0,1.0,0.0,0.0


In [16]:
## adding the current status of ess as a context variable.
context_data = context_data.sort_index()
ess_info = []
for session in tqdm(context_data.session.unique().tolist()):
    df = context_data[context_data['session']== session].copy()
    df['ess_status'] = 0
    if df['Label'].str.contains('ESS').any():
        ess_index = df.index[df['Label'] == 'car/ESS/on']
        for idx in reversed(ess_index):
            df.loc[:idx, 'ess_status'] = 0  # Set 'ess_status' to 0 for rows before the 'car/ESS/on' row
            df.loc[idx+1:, 'ess_status'] = 1
    ess_info.append(df)
data_ess = pd.concat(ess_info, axis=0)

100%|██████████| 1704/1704 [00:18<00:00, 91.70it/s] 


In [17]:
data_ess[data_ess['session']==205][['Label', 'session', 'ess_status']].dropna()

,Label,session,ess_status
146086,media/selectedSource/Radio,205.0,0
146205,car/driveMode/1.0,205.0,0
146621,car/charismaLevel/change,205.0,0
146634,car/charismaLevel/change,205.0,0
146831,car/ESS/on,205.0,0
147046,car/ESS/on,205.0,1
147122,navi/Start/Address,205.0,1
147354,car/driveMode/0.0,205.0,1
147392,media/selectedSource/Radio,205.0,1
147566,media/selectedSource/Bluetooth,205.0,1


In [18]:
data_ess[data_ess['session']==1645][['Label', 'session', 'ess_status']].dropna()

,Label,session,ess_status
3697821,car/driveMode/3.0,1645.0,0
3698358,car/driveMode/0.0,1645.0,0
3698581,car/driveMode/3.0,1645.0,0
3699191,car/driveMode/0.0,1645.0,0
3699399,car/driveMode/3.0,1645.0,0
3699483,media/selectedSource/Bluetooth,1645.0,0
3699484,media/selectedSource/Radio,1645.0,0
3699840,car/driveMode/0.0,1645.0,0


In [19]:
test_session = [1645]

drive_mode_info = []
for session in tqdm(data_ess.session.unique().tolist()):
# for session in tqdm(test_session):
    df = data_ess[data_ess['session']== session].copy()
    df = df.sort_values(by=['datetime'])
    df['current_drive_mode'] = 'car/driveMode/0.0'
    drive_modes = {'car/driveMode/0.0', 'car/driveMode/1.0', 'car/driveMode/2.0', 'car/driveMode/3.0'}

    # Iterate over DataFrame
    first_drive_mode_interaction = 0
    if df['Label'].str.contains('driveMode').any():
        for i, row in df.iterrows():
            label = row['Label']
            if label in drive_modes:
                if first_drive_mode_interaction ==0 and label == 'car/driveMode/0.0':
                    df.loc[:i, 'current_drive_mode'] = 'car/driveMode/3.0'
                # df.at[i, 'current_drive_mode'] = label
                first_drive_mode_interaction=1
                df.loc[i+1:, 'current_drive_mode'] = label
            else:
                continue
    drive_mode_info.append(df)
data_drivemode = pd.concat(drive_mode_info, axis=0)

100%|██████████| 1704/1704 [03:14<00:00,  8.78it/s]


In [20]:
data_drivemode[data_drivemode['session']==1645][['Label', 'session', 'current_drive_mode']].dropna()

,Label,session,current_drive_mode
3697821,car/driveMode/3.0,1645.0,car/driveMode/0.0
3698358,car/driveMode/0.0,1645.0,car/driveMode/3.0
3698581,car/driveMode/3.0,1645.0,car/driveMode/0.0
3699191,car/driveMode/0.0,1645.0,car/driveMode/3.0
3699399,car/driveMode/3.0,1645.0,car/driveMode/0.0
3699483,media/selectedSource/Bluetooth,1645.0,car/driveMode/3.0
3699484,media/selectedSource/Radio,1645.0,car/driveMode/3.0
3699840,car/driveMode/0.0,1645.0,car/driveMode/3.0


In [21]:
data_ess[data_ess['session']==16]

,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,time_second,distance_driven,car_id,month,weekday,hour,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,ess_status
4187,16.0,2022-09-09 17:48:13,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,9.0,4.0,17.0,0.0,1.0,0.0,0
4188,16.0,2022-09-09 17:48:14,NaN,0.0,0.0,0.0,0.0,24.0,0.0,0.0,...,1.0,11065.0,1.0,9.0,4.0,17.0,0.0,1.0,0.0,0
4189,16.0,2022-09-09 17:48:15,NaN,0.0,0.0,22.0,0.0,24.0,0.0,0.0,...,2.0,11065.0,1.0,9.0,4.0,17.0,0.0,1.0,0.0,0
4190,16.0,2022-09-09 17:48:16,NaN,0.0,0.0,22.0,0.0,24.0,26.0,0.0,...,3.0,11065.0,1.0,9.0,4.0,17.0,0.0,1.0,0.0,0
4191,16.0,2022-09-09 17:48:16,NaN,0.0,0.0,22.0,0.0,24.0,26.0,0.0,...,3.0,11065.0,1.0,9.0,4.0,17.0,0.0,1.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7623,16.0,2022-09-09 18:45:29,NaN,0.0,0.0,20.5,3200.0,852.0,26.5,0.0,...,3436.0,11114.0,1.0,9.0,4.0,18.0,0.0,1.0,0.0,0
7624,16.0,2022-09-09 18:45:30,NaN,0.0,0.0,20.5,3200.0,852.0,26.5,0.0,...,3437.0,11114.0,1.0,9.0,4.0,18.0,0.0,1.0,0.0,0
7625,16.0,2022-09-09 18:45:31,NaN,0.0,0.0,20.5,3200.0,852.0,26.5,0.0,...,3438.0,11114.0,1.0,9.0,4.0,18.0,0.0,1.0,0.0,0
7626,16.0,2022-09-09 18:45:32,NaN,0.0,0.0,20.5,3200.0,852.0,26.5,0.0,...,3439.0,11114.0,1.0,9.0,4.0,18.0,0.0,1.0,0.0,0


In [22]:
data_drivemode[data_drivemode['session']==16][['Label', 'session', 'current_drive_mode']].dropna()

,Label,session,current_drive_mode
4420,media/selectedSource/Radio,16.0,car/driveMode/0.0
4421,media/selectedSource/Radio,16.0,car/driveMode/0.0
5035,media/selectedSource/Bluetooth,16.0,car/driveMode/0.0
6338,media/selectedSource/Radio,16.0,car/driveMode/0.0
7484,media/selectedSource/Radio,16.0,car/driveMode/0.0


In [23]:
data_drivemode.Label.unique().tolist()

[nan,
 'media/selectedSource/Bluetooth',
 'media/selectedSource/Radio',
 'media/selectedSource/Favorite',
 'navi/Start/Address',
 'navi/Start/Favorite',
 'phone/goTo/Favorite',
 'car/driveMode/0.0',
 'car/driveMode/2.0',
 'phone/Start/CarPlay',
 'clima/AC/on',
 'car/driveMode/1.0',
 'car/charismaLevel/change',
 'car/ESS/on',
 'car/driveMode/3.0',
 'phone/Start/AndroidAuto',
 'clima/AC/off',
 'phone/Connect/NewDevice',
 'phone/Call/Favorite',
 'phone/Call/PersonX',
 'clima/AC/ECO',
 'car/Start/ParkAssistant',
 'media/selectedSource/CarPlay']

In [24]:
data_drivemode.current_drive_mode.unique().tolist()

['car/driveMode/0.0',
 'car/driveMode/3.0',
 'car/driveMode/2.0',
 'car/driveMode/1.0']

In [25]:
data_drivemode.dropna()[data_drivemode.dropna()['Label'].str.contains('clima')]['Label'].unique().tolist()

['clima/AC/on', 'clima/AC/off', 'clima/AC/ECO']

In [26]:
# import matplotlib.pyplot as plt

# df = data_drivemode[['temperature_out', 'temperature_in', 'month', 'car_id']]
# df['month'] = pd.Categorical(df['month'], categories=df['month'].unique().tolist(), ordered=True)
# averages = df.groupby(['car_id', 'month']).mean().reset_index()

# # Plotting
# fig, axes = plt.subplots(nrows=len(df['car_id'].unique()), ncols=1, figsize=(10, 6 * len(df['car_id'].unique())))

# for i, (car_id, group) in enumerate(averages.groupby('car_id')):
#     ax = axes[i]
#     group.plot(x='month', y=['temperature_out', 'temperature_in'], kind='bar', ax=ax, title=f'Car ID: {car_id}', legend=True)
#     ax.set_ylabel('Temperature')
#     ax.set_xlabel('Month')
#     ax.legend(['Temperature Out', 'Temperature In'])

# plt.tight_layout()
# plt.show()

In [27]:
len(data_drivemode['session'].unique().tolist())

1704

In [28]:
data_drivemode.dropna()[data_drivemode.dropna()['Label'].str.contains('clima')].dropna()

,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,distance_driven,car_id,month,weekday,hour,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,ess_status,current_drive_mode
123718,192.0,2022-12-27 21:37:25,clima/AC/on,0.000000,0.000000,5.0,0.000000,0.000000,19.000000,0.000000,...,14604.0,1.0,12.0,1.0,21.0,1.0,1.0,0.0,0,car/driveMode/0.0
229147,387.0,2023-04-05 10:18:00,clima/AC/off,186.359989,0.000000,3.0,15200.000000,6126.000000,26.500000,0.000000,...,22267.0,1.0,4.0,2.0,10.0,0.0,0.0,0.0,0,car/driveMode/3.0
511269,509.0,2023-04-27 13:33:32,clima/AC/off,310.000000,0.000000,16.0,101200.000000,6126.000000,30.000000,68.480000,...,24107.0,1.0,4.0,3.0,13.0,0.0,0.0,0.0,0,car/driveMode/0.0
924794,735.0,2023-05-31 08:03:30,clima/AC/off,130.000000,10.908400,17.0,9200.000000,4752.000000,23.701887,30.663599,...,26980.0,1.0,5.0,2.0,8.0,0.0,0.0,0.0,0,car/driveMode/0.0
1086117,766.0,2023-06-04 19:43:36,clima/AC/off,30.000000,31.922099,20.0,6400.000000,2828.517589,28.000000,12.238004,...,27591.0,1.0,6.0,6.0,19.0,0.0,1.0,1.0,0,car/driveMode/0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9637092,5086.0,2023-01-20 11:32:39,clima/AC/ECO,31.480222,0.000000,1.0,21194.333573,6126.000000,23.000000,142.080000,...,16400.0,6.0,1.0,4.0,11.0,1.0,0.0,1.0,0,car/driveMode/0.0
9859588,5169.0,2023-02-05 01:13:43,clima/AC/on,0.000000,36.552309,0.5,0.000000,0.000000,9.000000,15.360000,...,18822.0,6.0,2.0,6.0,1.0,1.0,0.0,0.0,0,car/driveMode/0.0
9916990,5246.0,2023-02-23 18:55:01,clima/AC/ECO,0.000000,0.000000,12.0,0.000000,102.000000,25.000000,0.000000,...,19744.0,6.0,2.0,3.0,18.0,1.0,0.0,0.0,0,car/driveMode/0.0
9916991,5246.0,2023-02-23 18:55:01,clima/AC/ECO,0.000000,0.000000,12.0,0.000000,102.000000,25.000000,0.000000,...,19744.0,6.0,2.0,3.0,18.0,1.0,0.0,0.0,0,car/driveMode/0.0


In [29]:
data_drivemode[data_drivemode['session']==387].sort_index().dropna()

,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,distance_driven,car_id,month,weekday,hour,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,ess_status,current_drive_mode
226222,387.0,2023-04-05 09:29:17,media/selectedSource/Radio,4.998313,5.171941,2.0,2155.106528,613.837251,2.0,37.837598,...,22245.0,1.0,4.0,2.0,9.0,0.0,0.0,0.0,0,car/driveMode/0.0
226280,387.0,2023-04-05 09:30:17,media/selectedSource/Radio,20.000000,1.756541,1.5,6776.912524,3968.346312,2.5,46.293333,...,22246.0,1.0,4.0,2.0,9.0,0.0,0.0,0.0,0,car/driveMode/0.0
226282,387.0,2023-04-05 09:30:17,media/selectedSource/Radio,20.000000,1.756541,1.5,6776.912524,3968.346312,2.5,46.293333,...,22246.0,1.0,4.0,2.0,9.0,0.0,0.0,0.0,0,car/driveMode/0.0
227035,387.0,2023-04-05 09:42:50,media/selectedSource/Radio,147.072876,195.389886,3.0,11731.272373,3546.139115,9.5,21.760000,...,22248.0,1.0,4.0,2.0,9.0,0.0,0.0,0.0,0,car/driveMode/0.0
227036,387.0,2023-04-05 09:42:50,media/selectedSource/Radio,147.072876,195.389886,3.0,11731.272373,3546.139115,9.5,21.760000,...,22248.0,1.0,4.0,2.0,9.0,0.0,0.0,0.0,0,car/driveMode/0.0
227292,387.0,2023-04-05 09:47:06,car/driveMode/3.0,0.000000,5.000000,3.0,136.094168,256.455538,13.0,49.358964,...,22249.0,1.0,4.0,2.0,9.0,0.0,0.0,0.0,0,car/driveMode/0.0
227301,387.0,2023-04-05 09:47:15,media/selectedSource/Radio,0.000000,0.000000,3.0,0.000000,0.000000,13.0,62.478505,...,22249.0,1.0,4.0,2.0,9.0,0.0,0.0,0.0,0,car/driveMode/3.0
227302,387.0,2023-04-05 09:47:15,media/selectedSource/Radio,0.000000,0.000000,3.0,0.000000,0.000000,13.0,62.478505,...,22249.0,1.0,4.0,2.0,9.0,0.0,0.0,0.0,0,car/driveMode/3.0
229147,387.0,2023-04-05 10:18:00,clima/AC/off,186.359989,0.000000,3.0,15200.000000,6126.000000,26.5,0.000000,...,22267.0,1.0,4.0,2.0,10.0,0.0,0.0,0.0,0,car/driveMode/3.0
229532,387.0,2023-04-05 10:24:27,media/selectedSource/Radio,161.330409,0.413250,4.0,21600.000000,5726.367370,26.5,62.080000,...,22272.0,1.0,4.0,2.0,10.0,0.0,0.0,0.0,0,car/driveMode/3.0


In [30]:
# test_session = [387]

# clima_mode_info = []
# # temp_delta = []
# for session in tqdm(data_drivemode.session.unique().tolist()):
# # for session in tqdm(test_session):
#     df = data_drivemode[data_drivemode['session']== session].copy().sort_index().reset_index()
#     filtered_df = df.dropna()[df.dropna()['Label'].str.contains('clima')].dropna()
#     if filtered_df.empty:
#         # print('empty')
#         df['current_clima_mode']  = 'unknown'
#     else:
#         df = df.sort_values(by=['datetime'])
#         # df['current_clima_mode'] = 'clima/AC/off'
#         clima_modes = {'clima/AC/on', 'clima/AC/off', 'clima/AC/ECO'}

#         # Iterate over DataFrame
#         first_clima_interaction_index = 0
#         for i, row in df.iterrows():
#             label = row['Label']
#             if label in clima_modes:
#                 if first_clima_interaction_index == 0:
#                     first_clima_interaction_index = i
#                 # df.at[i, 'current_clima_mode'] = label
#                 df.loc[i+1:, 'current_clima_mode'] = label
#             else:
#                 continue

#         if first_clima_interaction_index is not None:
#             temp1_index = df[df['temperature_in'] != 0].index.min()
#             temp1 = df.at[temp1_index, 'temperature_in']
#             temp2 = df['temperature_in'].iloc[first_clima_interaction_index - 1]
#             # print(temp1-temp2)
#             if abs(float(temp1) - float(temp2)) > 3:
#                 # print('in1')
#                 if df.at[first_clima_interaction_index, 'Label'] == 'clima/AC/ECO':
#                     df.loc[:first_clima_interaction_index - 1, 'current_clima_mode']  = 'clima/AC/on'
#                 elif df.at[first_clima_interaction_index, 'Label'] == 'clima/AC/on':
#                     df.loc[:first_clima_interaction_index - 1, 'current_clima_mode']  = 'clima/AC/ECO'
#                 elif df.at[first_clima_interaction_index, 'Label'] == 'clima/AC/off':
#                     df.loc[:first_clima_interaction_index - 1, 'current_clima_mode']  = 'clima/AC/ECO'
#             else:
#                 df.loc[:first_clima_interaction_index - 1, 'current_clima_mode'] = 'clima/AC/off'
#     clima_mode_info.append(df)
# data_clima = pd.concat(clima_mode_info, axis=0)

test_session = [387]

clima_mode_info = []
# temp_delta = []
for session in tqdm(data_drivemode.session.unique().tolist()):
# for session in tqdm(test_session):
    df = data_drivemode[data_drivemode['session']== session].copy().sort_index().reset_index()
    df['current_clima_mode']  = 'unknown'
    df = df.sort_values(by=['datetime'])
        # df['current_clima_mode'] = 'clima/AC/off'
    clima_modes = {'clima/AC/on', 'clima/AC/off', 'clima/AC/ECO'}
    if df['Label'].str.contains('clima').any():
        for i, row in df.iterrows():
            label = row['Label']
            if label in clima_modes:
                df.loc[i+1:, 'current_clima_mode'] = label
    clima_mode_info.append(df)
data_clima = pd.concat(clima_mode_info, axis=0)

100%|██████████| 1704/1704 [00:45<00:00, 37.77it/s]


In [31]:
data_clima

,index,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,...,car_id,month,weekday,hour,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,ess_status,current_drive_mode,current_clima_mode
0,0,7.0,2022-09-07 20:08:33,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,1.0,9.0,2.0,20.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown
1,1,7.0,2022-09-07 20:08:34,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,1.0,9.0,2.0,20.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown
2,2,7.0,2022-09-07 20:08:35,NaN,0.000000,0.000000,25.0,0.0,0.000000,0.0,...,1.0,9.0,2.0,20.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown
3,3,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,27.0,...,1.0,9.0,2.0,20.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown
4,4,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,27.0,...,1.0,9.0,2.0,20.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,10124879,5399.0,2023-03-31 17:09:13,NaN,53.755324,0.000000,13.5,17600.0,6126.000000,28.5,...,6.0,3.0,4.0,17.0,1.0,1.0,1.0,0,car/driveMode/0.0,unknown
2067,10124880,5399.0,2023-03-31 17:09:14,NaN,50.000000,0.000000,13.5,17200.0,5920.781157,28.5,...,6.0,3.0,4.0,17.0,1.0,1.0,1.0,0,car/driveMode/0.0,unknown
2068,10124881,5399.0,2023-03-31 17:09:15,NaN,56.226313,15.000000,13.5,18000.0,5699.908827,28.5,...,6.0,3.0,4.0,17.0,1.0,1.0,1.0,0,car/driveMode/0.0,unknown
2069,10124882,5399.0,2023-03-31 17:09:16,NaN,60.000000,0.000000,13.5,17600.0,5437.218229,28.5,...,6.0,3.0,4.0,17.0,1.0,1.0,1.0,0,car/driveMode/0.0,unknown


In [32]:
data_clima.dropna()[data_clima.dropna()['Label'].str.contains('clima')].dropna()

,index,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,...,car_id,month,weekday,hour,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,ess_status,current_drive_mode,current_clima_mode
57,123718,192.0,2022-12-27 21:37:25,clima/AC/on,0.000000,0.000000,5.0,0.000000,0.000000,19.000000,...,1.0,12.0,1.0,21.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown
3029,229147,387.0,2023-04-05 10:18:00,clima/AC/off,186.359989,0.000000,3.0,15200.000000,6126.000000,26.500000,...,1.0,4.0,2.0,10.0,0.0,0.0,0.0,0,car/driveMode/3.0,unknown
338,511269,509.0,2023-04-27 13:33:32,clima/AC/off,310.000000,0.000000,16.0,101200.000000,6126.000000,30.000000,...,1.0,4.0,3.0,13.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown
1159,924794,735.0,2023-05-31 08:03:30,clima/AC/off,130.000000,10.908400,17.0,9200.000000,4752.000000,23.701887,...,1.0,5.0,2.0,8.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown
3201,1086117,766.0,2023-06-04 19:43:36,clima/AC/off,30.000000,31.922099,20.0,6400.000000,2828.517589,28.000000,...,1.0,6.0,6.0,19.0,0.0,1.0,1.0,0,car/driveMode/0.0,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080,9637092,5086.0,2023-01-20 11:32:39,clima/AC/ECO,31.480222,0.000000,1.0,21194.333573,6126.000000,23.000000,...,6.0,1.0,4.0,11.0,1.0,0.0,1.0,0,car/driveMode/0.0,clima/AC/ECO
178,9859588,5169.0,2023-02-05 01:13:43,clima/AC/on,0.000000,36.552309,0.5,0.000000,0.000000,9.000000,...,6.0,2.0,6.0,1.0,1.0,0.0,0.0,0,car/driveMode/0.0,unknown
1347,9916990,5246.0,2023-02-23 18:55:01,clima/AC/ECO,0.000000,0.000000,12.0,0.000000,102.000000,25.000000,...,6.0,2.0,3.0,18.0,1.0,0.0,0.0,0,car/driveMode/0.0,unknown
1348,9916991,5246.0,2023-02-23 18:55:01,clima/AC/ECO,0.000000,0.000000,12.0,0.000000,102.000000,25.000000,...,6.0,2.0,3.0,18.0,1.0,0.0,0.0,0,car/driveMode/0.0,clima/AC/ECO


In [33]:
data_clima.dropna()[data_clima.dropna()['Label'].str.contains('clima')][['session']].value_counts()

session
1174.0     8
3965.0     6
4899.0     4
4075.0     4
766.0      4
801.0      4
857.0      4
4734.0     2
4839.0     2
4874.0     2
1514.0     2
5081.0     2
5052.0     2
5086.0     2
941.0      2
5246.0     2
4889.0     1
3993.0     1
4841.0     1
5169.0     1
4003.0     1
192.0      1
3992.0     1
3978.0     1
387.0      1
2725.0     1
2032.0     1
1666.0     1
1403.0     1
1359.0     1
1293.0     1
1217.0     1
1189.0     1
735.0      1
509.0      1
5305.0     1
Name: count, dtype: int64

In [34]:
data_clima[data_clima['session']==801.0][['Label', 'session', 'temperature_in', 'current_clima_mode']].dropna()

,Label,session,temperature_in,current_clima_mode
144,media/selectedSource/Favorite,801.0,33.0,unknown
147,media/selectedSource/Radio,801.0,33.0,unknown
148,media/selectedSource/Radio,801.0,33.0,unknown
259,clima/AC/off,801.0,32.5,unknown
261,clima/AC/off,801.0,32.5,clima/AC/off
478,clima/AC/off,801.0,32.5,clima/AC/off
480,clima/AC/off,801.0,32.5,clima/AC/off
651,phone/goTo/Favorite,801.0,32.0,clima/AC/off
649,phone/goTo/Favorite,801.0,32.0,clima/AC/off
2864,navi/Start/Address,801.0,30.0,clima/AC/off


In [35]:
data_clima[data_clima['session']==4899.0][['Label', 'session', 'temperature_in', 'current_clima_mode']].dropna()

,Label,session,temperature_in,current_clima_mode
79,media/selectedSource/Radio,4899.0,7.000000,unknown
78,media/selectedSource/Radio,4899.0,7.000000,unknown
987,media/selectedSource/Radio,4899.0,15.500000,unknown
988,media/selectedSource/Radio,4899.0,15.500000,unknown
1329,clima/AC/ECO,4899.0,18.000000,unknown
...,...,...,...,...
12431,navi/Start/Address,4899.0,30.500000,clima/AC/ECO
13254,navi/Start/Favorite,4899.0,30.500000,clima/AC/ECO
13255,navi/Start/Favorite,4899.0,30.500000,clima/AC/ECO
13269,navi/Start/Address,4899.0,30.195433,clima/AC/ECO


In [36]:
data_clima[data_clima['session']==4075.0][['Label', 'session', 'temperature_in', 'current_clima_mode']].dropna()

,Label,session,temperature_in,current_clima_mode
0,clima/AC/on,4075.0,0.0,unknown
1,clima/AC/on,4075.0,0.0,clima/AC/on
2,clima/AC/on,4075.0,0.0,clima/AC/on
3,clima/AC/on,4075.0,0.0,clima/AC/on
41,phone/Start/CarPlay,4075.0,12.0,clima/AC/on
43,phone/Start/CarPlay,4075.0,12.0,clima/AC/on
142,phone/Start/CarPlay,4075.0,12.5,clima/AC/on
143,phone/Start/CarPlay,4075.0,12.5,clima/AC/on
200,phone/Start/CarPlay,4075.0,13.5,clima/AC/on
199,phone/Start/CarPlay,4075.0,13.5,clima/AC/on


In [37]:
data_clima[data_clima['session']==3965.0][['Label', 'session', 'temperature_in', 'current_clima_mode']].dropna()

,Label,session,temperature_in,current_clima_mode
112,phone/Start/CarPlay,3965.0,22.5,unknown
113,phone/Start/CarPlay,3965.0,22.5,unknown
4893,navi/Start/Favorite,3965.0,27.0,unknown
4894,navi/Start/Favorite,3965.0,27.0,unknown
4896,navi/Start/Address,3965.0,27.0,unknown
4897,navi/Start/Address,3965.0,27.0,unknown
5716,navi/Start/Favorite,3965.0,27.5,unknown
5717,navi/Start/Favorite,3965.0,27.5,unknown
5718,navi/Start/Address,3965.0,27.5,unknown
5719,navi/Start/Address,3965.0,27.5,unknown


In [38]:
data_clima[data_clima['session']==1174][['Label', 'session', 'temperature_in', 'current_clima_mode']].dropna()

,Label,session,temperature_in,current_clima_mode
1010,navi/Start/Favorite,1174.0,24.500000,unknown
1012,navi/Start/Address,1174.0,24.500000,unknown
1043,navi/Start/Address,1174.0,24.500000,unknown
2618,media/selectedSource/Radio,1174.0,25.000000,unknown
2632,clima/AC/off,1174.0,25.000000,unknown
3341,clima/AC/off,1174.0,26.086016,clima/AC/off
3357,clima/AC/off,1174.0,26.000000,clima/AC/off
3361,clima/AC/off,1174.0,26.000000,clima/AC/off
5733,clima/AC/off,1174.0,29.000000,clima/AC/off
6185,media/selectedSource/Radio,1174.0,29.000000,clima/AC/off


In [39]:
data_clima.dropna()[data_clima.dropna()['Label'].str.contains('selectedSource')].dropna()['Label'].unique()

array(['media/selectedSource/Bluetooth', 'media/selectedSource/Radio',
       'media/selectedSource/Favorite', 'media/selectedSource/CarPlay'],
      dtype=object)

In [40]:
data_clima[data_clima['session']==16].Label.dropna()

233         media/selectedSource/Radio
234         media/selectedSource/Radio
848     media/selectedSource/Bluetooth
2151        media/selectedSource/Radio
3297        media/selectedSource/Radio
Name: Label, dtype: object

In [41]:
test_session = [387]
# # data_clima = data_clima.drop(columns='current_clima_mode')
# df_clima = data_clima.sort_values(by=['session', 'datetime'])
# df_clima = df_clima.reset_index(drop=True)
# df_clima = df_clima.drop(columns=['index'])

media_source_info = []
for session in tqdm(data_clima.session.unique().tolist()):
    df = data_clima[data_clima['session']== session].copy().sort_values(by=['datetime'])
    df = df.reset_index(drop=True)
    df['current_media_source'] = 'media/selectedSource/unavailable'
    media_source = {'media/selectedSource/Bluetooth', 'media/selectedSource/Radio',
    'media/selectedSource/Favorite', 'media/selectedSource/CarPlay'}

    if df['Label'].str.contains('selectedSource').any():
        for i, row in df.iterrows():
            label = row['Label']
            if label in media_source:
                # df.at[i, 'current_media_source'] = label
                df.loc[i+1:, 'current_media_source'] = label
            else:
                continue
    media_source_info.append(df)
data_media = pd.concat(media_source_info, axis=0)

100%|██████████| 1704/1704 [06:37<00:00,  4.29it/s]


In [42]:
data_media[data_media['session']==387][['Label', 'session', 'current_media_source']].dropna()

,Label,session,current_media_source
104,media/selectedSource/Radio,387.0,media/selectedSource/unavailable
162,media/selectedSource/Radio,387.0,media/selectedSource/Radio
164,media/selectedSource/Radio,387.0,media/selectedSource/Radio
917,media/selectedSource/Radio,387.0,media/selectedSource/Radio
918,media/selectedSource/Radio,387.0,media/selectedSource/Radio
1174,car/driveMode/3.0,387.0,media/selectedSource/Radio
1183,media/selectedSource/Radio,387.0,media/selectedSource/Radio
1184,media/selectedSource/Radio,387.0,media/selectedSource/Radio
3029,clima/AC/off,387.0,media/selectedSource/Radio
3414,media/selectedSource/Radio,387.0,media/selectedSource/Radio


In [43]:
data_media.dropna()[data_media.dropna()['Label'].str.contains('selectedSource')].dropna()

,index,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,...,month,weekday,hour,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,ess_status,current_drive_mode,current_clima_mode,current_media_source
1627,1627,7.0,2022-09-07 21:00:51,media/selectedSource/Bluetooth,0.000000,44.108063,19.0,0.000000,0.000000,26.5,...,9.0,2.0,21.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
233,4420,16.0,2022-09-09 17:52:06,media/selectedSource/Radio,0.000000,7.341252,19.5,800.000000,877.203437,25.5,...,9.0,4.0,17.0,0.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
234,4421,16.0,2022-09-09 17:52:06,media/selectedSource/Radio,0.000000,7.341252,19.5,800.000000,877.203437,25.5,...,9.0,4.0,17.0,0.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/Radio
848,5035,16.0,2022-09-09 18:02:19,media/selectedSource/Bluetooth,20.000000,0.000000,21.0,2400.000000,1416.000000,26.5,...,9.0,4.0,18.0,0.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/Radio
2151,6338,16.0,2022-09-09 18:24:03,media/selectedSource/Radio,0.000000,0.000000,21.0,1600.000000,354.000000,27.5,...,9.0,4.0,18.0,0.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/Bluetooth
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110,10114077,5385.0,2023-03-28 09:06:58,media/selectedSource/Radio,0.000000,79.044432,0.0,5995.880402,4277.195481,0.0,...,3.0,1.0,9.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
111,10114078,5385.0,2023-03-28 09:06:58,media/selectedSource/Radio,0.000000,79.044432,0.0,5995.880402,4277.195481,0.0,...,3.0,1.0,9.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/Radio
425,10115142,5386.0,2023-03-28 10:10:12,media/selectedSource/Radio,195.419114,0.000000,4.0,76395.061828,6126.000000,27.5,...,3.0,1.0,10.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
1662,10120436,5389.0,2023-03-28 17:22:34,media/selectedSource/Radio,50.000000,1.006737,8.0,8800.000000,6073.053724,25.5,...,3.0,1.0,17.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable


In [44]:
data_media.dropna()[data_media.dropna()['Label'].str.contains('navi')].dropna()

,index,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,...,month,weekday,hour,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,ess_status,current_drive_mode,current_clima_mode,current_media_source
182,13610,33.0,2022-09-12 17:09:43,navi/Start/Favorite,0.0,0.000000,23.5,1600.000000,3221.903447,29.115677,...,9.0,0.0,17.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
183,13611,33.0,2022-09-12 17:09:43,navi/Start/Address,0.0,0.000000,23.5,1600.000000,3221.903447,29.115677,...,9.0,0.0,17.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
1775,21482,35.0,2022-09-13 07:43:55,navi/Start/Favorite,150.0,0.000000,16.0,1600.000000,1416.000000,24.606187,...,9.0,1.0,7.0,1.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
1776,21483,35.0,2022-09-13 07:43:55,navi/Start/Address,150.0,0.000000,16.0,1600.000000,1416.000000,24.606187,...,9.0,1.0,7.0,1.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
784,23812,40.0,2022-09-14 18:09:11,navi/Start/Favorite,0.0,0.000000,20.0,1600.000000,1195.217862,25.000000,...,9.0,2.0,18.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,10111789,5364.0,2023-03-23 08:24:12,navi/Start/Favorite,0.0,3.955310,13.5,1600.000000,512.863260,21.000000,...,3.0,3.0,8.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/Radio
727,10111790,5364.0,2023-03-23 08:24:12,navi/Start/Favorite,0.0,3.955310,13.5,1600.000000,512.863260,21.000000,...,3.0,3.0,8.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/Radio
728,10111791,5364.0,2023-03-23 08:24:12,navi/Start/Address,0.0,3.955310,13.5,1600.000000,512.863260,21.000000,...,3.0,3.0,8.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/Radio
729,10111792,5364.0,2023-03-23 08:24:12,navi/Start/Address,0.0,3.955310,13.5,1600.000000,512.863260,21.000000,...,3.0,3.0,8.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/Radio


In [45]:
data_media.dropna()[data_media.dropna()['Label'].str.contains('navi')].dropna()['Label'].unique().tolist()

['navi/Start/Favorite', 'navi/Start/Address']

In [46]:
data_media

,index,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,...,month,weekday,hour,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,ess_status,current_drive_mode,current_clima_mode,current_media_source
0,0,7.0,2022-09-07 20:08:33,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,9.0,2.0,20.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
1,1,7.0,2022-09-07 20:08:34,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,9.0,2.0,20.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
2,2,7.0,2022-09-07 20:08:35,NaN,0.000000,0.000000,25.0,0.0,0.000000,0.0,...,9.0,2.0,20.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
3,3,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,27.0,...,9.0,2.0,20.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
4,4,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,27.0,...,9.0,2.0,20.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,10124879,5399.0,2023-03-31 17:09:13,NaN,53.755324,0.000000,13.5,17600.0,6126.000000,28.5,...,3.0,4.0,17.0,1.0,1.0,1.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
2067,10124880,5399.0,2023-03-31 17:09:14,NaN,50.000000,0.000000,13.5,17200.0,5920.781157,28.5,...,3.0,4.0,17.0,1.0,1.0,1.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
2068,10124881,5399.0,2023-03-31 17:09:15,NaN,56.226313,15.000000,13.5,18000.0,5699.908827,28.5,...,3.0,4.0,17.0,1.0,1.0,1.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
2069,10124882,5399.0,2023-03-31 17:09:16,NaN,60.000000,0.000000,13.5,17600.0,5437.218229,28.5,...,3.0,4.0,17.0,1.0,1.0,1.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable


In [47]:
data_media = data_media.sort_values(by=['session', 'datetime'])
data_media = data_media.reset_index(drop=True)
data_media = data_media.drop(columns=['index'])
data_media

,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,month,weekday,hour,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,ess_status,current_drive_mode,current_clima_mode,current_media_source
0,7.0,2022-09-07 20:08:33,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,...,9.0,2.0,20.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
1,7.0,2022-09-07 20:08:34,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,...,9.0,2.0,20.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
2,7.0,2022-09-07 20:08:35,NaN,0.000000,0.000000,25.0,0.0,0.000000,0.0,0.00,...,9.0,2.0,20.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
3,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,27.0,0.00,...,9.0,2.0,20.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
4,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,27.0,0.00,...,9.0,2.0,20.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10124879,5399.0,2023-03-31 17:09:13,NaN,53.755324,0.000000,13.5,17600.0,6126.000000,28.5,92.48,...,3.0,4.0,17.0,1.0,1.0,1.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
10124880,5399.0,2023-03-31 17:09:14,NaN,50.000000,0.000000,13.5,17200.0,5920.781157,28.5,93.12,...,3.0,4.0,17.0,1.0,1.0,1.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
10124881,5399.0,2023-03-31 17:09:15,NaN,56.226313,15.000000,13.5,18000.0,5699.908827,28.5,93.44,...,3.0,4.0,17.0,1.0,1.0,1.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
10124882,5399.0,2023-03-31 17:09:16,NaN,60.000000,0.000000,13.5,17600.0,5437.218229,28.5,93.76,...,3.0,4.0,17.0,1.0,1.0,1.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable


In [48]:
test_session = [387]
navi_status = []

# Iterate over unique sessions
for session in tqdm(data_media['session'].unique()):
    # Subset dataframe for the current session45
    df = data_media[data_media['session'] == session].copy().reset_index()
    
    # Initialize 'nav_guidance_status' column
    df['nav_guidance_status'] = 'navi_inactive'

    # Define labels related to navigation
    navi_labels = {'navi/Start/Address', 'navi/Start/Favorite'}
    
    # Check if navigation-related labels exist in the session
    if df['Label'].str.contains('navi').any():
        # Iterate through rows of the session dataframe
        for i, row in df.iterrows():
            label = row['Label']
            if label in navi_labels:
                # Update 'nav_guidance_status' to indicate active navigation
                df.loc[i+1:, 'nav_guidance_status'] = 'navi_active'
                break
    navi_status.append(df)

# Concatenate dataframes for all sessions
data_navi = pd.concat(navi_status, axis=0)

100%|██████████| 1704/1704 [01:19<00:00, 21.37it/s]


In [49]:
data_media.dropna()[data_media.dropna()['Label'].str.contains('ParkAssistant')].dropna()

,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,month,weekday,hour,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,ess_status,current_drive_mode,current_clima_mode,current_media_source
3744985,1659.0,2022-12-22 10:32:00,car/Start/ParkAssistant,5.707657,0.0,9.5,3600.000000,1110.000000,25.5,0.00000,...,12.0,3.0,10.0,0.0,0.0,0.0,1,car/driveMode/0.0,unknown,media/selectedSource/Bluetooth
8594242,4589.0,2023-06-22 14:22:22,car/Start/ParkAssistant,63.394196,0.0,23.5,19836.577804,5138.451333,25.0,25.84526,...,6.0,3.0,14.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/Radio


In [50]:
test_session = [387]
park_assistants_status = []

# Iterate over unique sessions
for session in tqdm(data_navi['session'].unique()):
    # Subset dataframe for the current session
    df = data_navi[data_navi['session'] == session].copy().reset_index()
    
    # Initialize 'nav_guidance_status' column
    df['proximity_to_parking_spot'] = 'no_parking_spot_closeby'

    # Define labels related to navigation
    park_assistant = {'car/Start/ParkAssistant'}

    # df.dropna()[df.dropna()['Label'].str.contains('ParkAssistant')].dropna()
    # Check if navigation-related labels exist in the session
    if df['Label'].str.contains('ParkAssistant').any():
        print(session, 'i am in')
        # Iterate through rows of the session dataframe
        for i, row in df.iterrows():
            label = row['Label']
            if label in park_assistant:
                # Update 'nav_guidance_status' to indicate active navigation
                df.loc[i-window:i+window, 'proximity_to_parking_spot'] = 'parking_spot_closeby'
                break
    park_assistants_status.append(df)

# Concatenate dataframes for all sessions
data_parkassistant = pd.concat(park_assistants_status, axis=0)

 36%|███▌      | 611/1704 [00:07<00:14, 74.08it/s]

1659.0 i am in


 85%|████████▍ | 1441/1704 [00:18<00:03, 82.48it/s]

4589.0 i am in


100%|██████████| 1704/1704 [00:22<00:00, 76.94it/s]


In [51]:
data_parkassistant[data_parkassistant['session']==1659.0]['proximity_to_parking_spot'].unique()

array(['no_parking_spot_closeby', 'parking_spot_closeby'], dtype=object)

In [52]:
data_media.dropna()[data_media.dropna()['Label'].str.contains('phone')].dropna()

,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,month,weekday,hour,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,ess_status,current_drive_mode,current_clima_mode,current_media_source
34273,46.0,2022-09-16 11:36:04,phone/goTo/Favorite,20.00000,0.000000,13.0,12000.000000,3543.000000,24.5,8.426007,...,9.0,4.0,11.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
44552,150.0,2022-12-20 15:37:33,phone/Start/CarPlay,0.00000,53.388109,8.5,2400.000000,738.000000,24.5,15.040000,...,12.0,1.0,15.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/Radio
144772,203.0,2022-12-29 16:40:56,phone/goTo/Favorite,0.00000,203.264859,13.5,2800.000000,975.726155,20.5,6.400000,...,12.0,3.0,16.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
224365,385.0,2023-04-04 17:02:41,phone/Start/AndroidAuto,16.10772,0.000000,9.0,4000.000000,6126.000000,22.0,0.000000,...,4.0,1.0,17.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
420006,465.0,2023-04-19 12:18:40,phone/goTo/Favorite,40.00000,0.000000,12.0,16400.000000,4549.259395,17.0,0.000000,...,4.0,2.0,12.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10058296,5334.0,2023-03-13 20:05:05,phone/goTo/Favorite,0.00000,0.000000,19.0,0.000000,0.000000,24.5,9.600000,...,3.0,0.0,20.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable
10064080,5339.0,2023-03-14 19:04:57,phone/goTo/Favorite,0.00000,0.000000,9.5,400.000000,420.000000,16.5,122.880000,...,3.0,1.0,19.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/Radio
10066875,5341.0,2023-03-15 08:13:29,phone/goTo/Favorite,20.00000,0.000000,0.5,1200.000000,3173.297789,14.5,74.240000,...,3.0,2.0,8.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/Radio
10117200,5387.0,2023-03-28 12:54:10,phone/goTo/Favorite,270.00000,0.000000,8.5,30834.067022,6126.000000,26.5,140.160000,...,3.0,1.0,12.0,0.0,0.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable


In [53]:
data_media.dropna()[data_media.dropna()['Label'].str.contains('phone')].dropna()['Label'].unique()

array(['phone/goTo/Favorite', 'phone/Start/CarPlay',
       'phone/Start/AndroidAuto', 'phone/Connect/NewDevice',
       'phone/Call/Favorite', 'phone/Call/PersonX'], dtype=object)

In [54]:
data_parkassistant

,level_0,index,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,...,hour,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,ess_status,current_drive_mode,current_clima_mode,current_media_source,nav_guidance_status,proximity_to_parking_spot
0,0,0,7.0,2022-09-07 20:08:33,NaN,0.000000,0.000000,0.0,0.0,0.000000,...,20.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby
1,1,1,7.0,2022-09-07 20:08:34,NaN,0.000000,0.000000,0.0,0.0,0.000000,...,20.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby
2,2,2,7.0,2022-09-07 20:08:35,NaN,0.000000,0.000000,25.0,0.0,0.000000,...,20.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby
3,3,3,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,...,20.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby
4,4,4,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,...,20.0,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,2066,10124879,5399.0,2023-03-31 17:09:13,NaN,53.755324,0.000000,13.5,17600.0,6126.000000,...,17.0,1.0,1.0,1.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby
2067,2067,10124880,5399.0,2023-03-31 17:09:14,NaN,50.000000,0.000000,13.5,17200.0,5920.781157,...,17.0,1.0,1.0,1.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby
2068,2068,10124881,5399.0,2023-03-31 17:09:15,NaN,56.226313,15.000000,13.5,18000.0,5699.908827,...,17.0,1.0,1.0,1.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby
2069,2069,10124882,5399.0,2023-03-31 17:09:16,NaN,60.000000,0.000000,13.5,17600.0,5437.218229,...,17.0,1.0,1.0,1.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby


In [10]:
# context_data = pd.read_csv('../data/06_context_feat_engg/data_featue_engineering.csv', parse_dates=['datetime'], index_col=0, low_memory=False)

In [12]:
test_session = [387]
phone_status = []

data_parkassistant = data_parkassistant.sort_values(by=['session', 'datetime'])
data_parkassistant = data_parkassistant.reset_index(drop=True)
# data_parkassistant = data_parkassistant.drop(columns=['level_0', 'index'])

for session in tqdm(data_parkassistant['session'].unique()):

    df = data_parkassistant[data_parkassistant['session'] == session].copy().reset_index()
    
    df['phone_status'] = 'unconnected'

    phone_labels = {'phone/goTo/Favorite', 'phone/Start/CarPlay',
       'phone/Start/AndroidAuto', 'phone/Connect/NewDevice',
       'phone/Call/Favorite', 'phone/Call/PersonX'}
    
    if df['Label'].str.contains('phone').any():

        for i, row in df.iterrows():
            label = row['Label']
            if label in phone_labels:
                df.loc[i-window:i, 'phone_status'] = 'phone_connected'
    phone_status.append(df)

data_phone_status = pd.concat(phone_status, axis=0)

100%|██████████| 1704/1704 [07:58<00:00,  3.56it/s]


In [17]:
# data_phone_status = data_phone_status.drop(columns=['index']).reset_index(drop=True)
# data_phone_status.to_csv('../data/06_context_feat_engg/data_featue_engineering.csv')

In [56]:
data_phone_status[data_phone_status['session']==46]['phone_status'].unique()

array(['unconnected', 'phone_connected'], dtype=object)

In [57]:
data_phone_status

,index,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,...,seatbelt_codriver,seatbelt_rear_l,seatbelt_rear_r,ess_status,current_drive_mode,current_clima_mode,current_media_source,nav_guidance_status,proximity_to_parking_spot,phone_status
0,0,7.0,2022-09-07 20:08:33,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected
1,1,7.0,2022-09-07 20:08:34,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.0,...,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected
2,2,7.0,2022-09-07 20:08:35,NaN,0.000000,0.000000,25.0,0.0,0.000000,0.0,...,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected
3,3,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,27.0,...,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected
4,4,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,27.0,...,1.0,1.0,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,10124879,5399.0,2023-03-31 17:09:13,NaN,53.755324,0.000000,13.5,17600.0,6126.000000,28.5,...,1.0,1.0,1.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected
2067,10124880,5399.0,2023-03-31 17:09:14,NaN,50.000000,0.000000,13.5,17200.0,5920.781157,28.5,...,1.0,1.0,1.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected
2068,10124881,5399.0,2023-03-31 17:09:15,NaN,56.226313,15.000000,13.5,18000.0,5699.908827,28.5,...,1.0,1.0,1.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected
2069,10124882,5399.0,2023-03-31 17:09:16,NaN,60.000000,0.000000,13.5,17600.0,5437.218229,28.5,...,1.0,1.0,1.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected


In [58]:
test_session = [387]
bluetooth_device_status = []

df_phone_status = data_phone_status.sort_values(by=['session', 'datetime'])
df_phone_status = df_phone_status.reset_index(drop=True)
df_phone_status = df_phone_status.drop(columns=['index'])

for session in tqdm(data_phone_status['session'].unique()):
    df = data_phone_status[data_phone_status['session'] == session].copy().reset_index()
    df['bluetooth_connected'] = 'bluetooth_unconnected'
    phone_labels = {'media/selectedSource/Bluetooth'}
    if df['Label'].str.contains('Bluetooth|phone').any():
        for i, row in df.iterrows():
            label = row['Label']
            if label in phone_labels:
                df.loc[i-window:, 'bluetooth_connected'] = 'bluetooth_connected'
    bluetooth_device_status.append(df)

data_bluetooth_status = pd.concat(bluetooth_device_status, axis=0)

100%|██████████| 1704/1704 [04:09<00:00,  6.83it/s]


In [59]:
south_germany_season_mapping = {
    1: 'Winter',
    2: 'Winter',
    3: 'Spring',
    4: 'Spring',
    5: 'Spring',
    6: 'Summer',
    7: 'Summer',
    8: 'Summer',
    9: 'Autumn',
    10: 'Autumn',
    11: 'Autumn',
    12: 'Winter'
}

data_bluetooth_status['season'] = data_bluetooth_status['month'].map(south_germany_season_mapping)

In [60]:
data_bluetooth_status.dropna()[data_bluetooth_status.dropna()['Label'].str.contains('CarPlay')].dropna()

,level_0,index,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,...,seatbelt_rear_r,ess_status,current_drive_mode,current_clima_mode,current_media_source,nav_guidance_status,proximity_to_parking_spot,phone_status,bluetooth_connected,season
1835,1835,44552,150.0,2022-12-20 15:37:33,phone/Start/CarPlay,0.0,53.388109,8.5,2400.000000,738.000000,...,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/Radio,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Winter
4158,4158,4424458,1935.0,2023-02-12 14:17:14,media/selectedSource/CarPlay,0.0,0.000000,10.0,2400.000000,4656.000000,...,1.0,0,car/driveMode/3.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Winter
4159,4159,4424459,1935.0,2023-02-12 14:17:14,media/selectedSource/CarPlay,0.0,0.000000,10.0,2400.000000,4656.000000,...,1.0,0,car/driveMode/3.0,unknown,media/selectedSource/CarPlay,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Winter
240,240,5206758,2389.0,2022-09-10 21:30:00,phone/Start/CarPlay,0.0,3.129767,15.0,0.000000,0.000000,...,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn
2032,2032,5823678,2708.0,2022-11-18 15:23:19,media/selectedSource/CarPlay,10.0,0.000000,8.5,5200.000000,726.540844,...,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,2106,10041462,5330.0,2023-03-13 10:17:32,media/selectedSource/CarPlay,50.0,0.000000,13.0,17200.000000,6126.000000,...,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/Radio,navi_active,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring
3951,3951,10043307,5330.0,2023-03-13 10:48:17,media/selectedSource/CarPlay,40.0,0.000000,11.5,15767.936863,4683.847576,...,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/Radio,navi_active,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring
394,394,10048979,5331.0,2023-03-13 13:51:18,media/selectedSource/CarPlay,110.0,5.000000,17.0,37386.293552,6126.000000,...,0.0,0,car/driveMode/3.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring
116,116,10055037,5333.0,2023-03-13 18:19:42,media/selectedSource/CarPlay,0.0,5.000000,19.5,2000.000000,2625.849453,...,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/Radio,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring


In [61]:
data_bluetooth_status

,level_0,index,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,...,seatbelt_rear_r,ess_status,current_drive_mode,current_clima_mode,current_media_source,nav_guidance_status,proximity_to_parking_spot,phone_status,bluetooth_connected,season
0,0,0,7.0,2022-09-07 20:08:33,NaN,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn
1,1,1,7.0,2022-09-07 20:08:34,NaN,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn
2,2,2,7.0,2022-09-07 20:08:35,NaN,0.000000,0.000000,25.0,0.0,0.000000,...,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn
3,3,3,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,...,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn
4,4,4,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,...,0.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2066,2066,10124879,5399.0,2023-03-31 17:09:13,NaN,53.755324,0.000000,13.5,17600.0,6126.000000,...,1.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring
2067,2067,10124880,5399.0,2023-03-31 17:09:14,NaN,50.000000,0.000000,13.5,17200.0,5920.781157,...,1.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring
2068,2068,10124881,5399.0,2023-03-31 17:09:15,NaN,56.226313,15.000000,13.5,18000.0,5699.908827,...,1.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring
2069,2069,10124882,5399.0,2023-03-31 17:09:16,NaN,60.000000,0.000000,13.5,17600.0,5437.218229,...,1.0,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring


In [62]:
# data_bluetooth_status.to_csv('saved_halfway.csv')

In [63]:
test_session = [387]
phone_os = []

df_bluetooth_status = data_bluetooth_status.sort_values(by=['session', 'datetime'])
df_bluetooth_status = df_bluetooth_status.reset_index(drop=True)
df_bluetooth_status = df_bluetooth_status.drop(columns=['index'])

phone_labels = {'phone/Start/CarPlay', 'media/selectedSource/CarPlay',  'phone/Start/AndroidAuto'}
Android = {'phone/Start/AndroidAuto'}
carplay = {'phone/Start/CarPlay', 'media/selectedSource/CarPlay'}
others = {'phone/goTo/Favorite', 'phone/Connect/NewDevice',
       'phone/Call/Favorite', 'phone/Call/PersonX', 'media/selectedSource/Bluetooth', }

phone_os = []

for session in tqdm(df_bluetooth_status['session'].unique()):
    df = df_bluetooth_status[df_bluetooth_status['session'] == session].copy()
    df['phone_os'] = 'unconnected'
    if df['Label'].str.contains('Bluetooth|phone|AndroidAuto|CarPlay').any():
        if df['Label'].str.contains('AndroidAuto').any():
            df['phone_os'] = 'Android'
        elif df['Label'].str.contains('CarPlay').any():
            df['phone_os'] = 'ios'
        else:
            df['phone_os'] = 'unknown'
    phone_os.append(df)

data_phone_os = pd.concat(phone_os, axis=0)

100%|██████████| 1704/1704 [00:22<00:00, 76.22it/s]


In [64]:
data_phone_os[data_phone_os['session']==16]

,level_0,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,...,ess_status,current_drive_mode,current_clima_mode,current_media_source,nav_guidance_status,proximity_to_parking_spot,phone_status,bluetooth_connected,season,phone_os
4187,0,16.0,2022-09-09 17:48:13,NaN,0.0,0.0,0.0,0.0,0.0,0.0,...,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown
4188,1,16.0,2022-09-09 17:48:14,NaN,0.0,0.0,0.0,0.0,24.0,0.0,...,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown
4189,2,16.0,2022-09-09 17:48:15,NaN,0.0,0.0,22.0,0.0,24.0,0.0,...,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown
4190,3,16.0,2022-09-09 17:48:16,NaN,0.0,0.0,22.0,0.0,24.0,26.0,...,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown
4191,4,16.0,2022-09-09 17:48:16,NaN,0.0,0.0,22.0,0.0,24.0,26.0,...,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7623,3436,16.0,2022-09-09 18:45:29,NaN,0.0,0.0,20.5,3200.0,852.0,26.5,...,0,car/driveMode/0.0,unknown,media/selectedSource/Radio,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_connected,Autumn,unknown
7624,3437,16.0,2022-09-09 18:45:30,NaN,0.0,0.0,20.5,3200.0,852.0,26.5,...,0,car/driveMode/0.0,unknown,media/selectedSource/Radio,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_connected,Autumn,unknown
7625,3438,16.0,2022-09-09 18:45:31,NaN,0.0,0.0,20.5,3200.0,852.0,26.5,...,0,car/driveMode/0.0,unknown,media/selectedSource/Radio,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_connected,Autumn,unknown
7626,3439,16.0,2022-09-09 18:45:32,NaN,0.0,0.0,20.5,3200.0,852.0,26.5,...,0,car/driveMode/0.0,unknown,media/selectedSource/Radio,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_connected,Autumn,unknown


In [65]:
sorted(data_phone_os[~data_phone_os.phone_os.isin(['unknown', 'unconnected'])][['Label', 'session', 'phone_os']].dropna().session.unique().tolist())

[150.0,
 385.0,
 490.0,
 502.0,
 592.0,
 832.0,
 937.0,
 1359.0,
 1935.0,
 2389.0,
 2708.0,
 2712.0,
 2844.0,
 2845.0,
 2849.0,
 3334.0,
 3404.0,
 3415.0,
 3953.0,
 3959.0,
 3965.0,
 3976.0,
 3978.0,
 3990.0,
 3992.0,
 4003.0,
 4012.0,
 4016.0,
 4018.0,
 4019.0,
 4027.0,
 4029.0,
 4030.0,
 4031.0,
 4038.0,
 4071.0,
 4075.0,
 4076.0,
 4078.0,
 4079.0,
 4083.0,
 4084.0,
 4086.0,
 4089.0,
 4093.0,
 4098.0,
 4106.0,
 4107.0,
 4115.0,
 4119.0,
 4122.0,
 4127.0,
 4139.0,
 4167.0,
 4183.0,
 4185.0,
 4197.0,
 4212.0,
 4219.0,
 4220.0,
 4222.0,
 4223.0,
 4226.0,
 4227.0,
 4228.0,
 4230.0,
 4237.0,
 4248.0,
 4261.0,
 4265.0,
 4266.0,
 4293.0,
 4296.0,
 4301.0,
 4317.0,
 4319.0,
 4323.0,
 4333.0,
 4344.0,
 4345.0,
 4346.0,
 4347.0,
 4349.0,
 4351.0,
 4352.0,
 4354.0,
 4360.0,
 4363.0,
 4364.0,
 4365.0,
 4367.0,
 4370.0,
 4371.0,
 4373.0,
 4375.0,
 4383.0,
 4384.0,
 4386.0,
 4388.0,
 4389.0,
 4390.0,
 4393.0,
 4396.0,
 4397.0,
 4398.0,
 4400.0,
 4404.0,
 4410.0,
 4415.0,
 4419.0,
 4420.0,
 4421.0,

In [66]:
data_phone_os[~data_phone_os.phone_os.isin(['unknown', 'unconnected'])][data_phone_os.session==502][['Label', 'session', 'phone_os']].dropna()

/var/folders/n8/1ntcg5gj7nvf5wh_s2w95yh80000gn/T/ipykernel_49473/1199504737.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  data_phone_os[~data_phone_os.phone_os.isin(['unknown', 'unconnected'])][data_phone_os.session==502][['Label', 'session', 'phone_os']].dropna()


,Label,session,phone_os
486922,media/selectedSource/Radio,502.0,Android
486950,phone/Start/AndroidAuto,502.0,Android
486951,media/selectedSource/Bluetooth,502.0,Android
486952,media/selectedSource/Bluetooth,502.0,Android
490685,phone/Start/AndroidAuto,502.0,Android
490708,phone/Start/AndroidAuto,502.0,Android
491131,media/selectedSource/Radio,502.0,Android
491132,media/selectedSource/Radio,502.0,Android
492430,media/selectedSource/Radio,502.0,Android
492947,media/selectedSource/Radio,502.0,Android


In [67]:
data_phone_os.dropna()[data_phone_os.dropna()['Label'].str.contains('NewDevice')].dropna()

,level_0,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,...,ess_status,current_drive_mode,current_clima_mode,current_media_source,nav_guidance_status,proximity_to_parking_spot,phone_status,bluetooth_connected,season,phone_os
2161654,38,1178.0,2022-09-14 20:22:41,phone/Connect/NewDevice,0.000000,0.000000,21.0,0.0,0.0,23.5,...,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_connected,Autumn,unknown
3744515,9202,1659.0,2022-12-22 10:24:10,phone/Connect/NewDevice,38.484954,0.000000,9.5,3600.0,1314.0,26.5,...,0,car/driveMode/0.0,unknown,media/selectedSource/Radio,navi_active,no_parking_spot_closeby,unconnected,bluetooth_connected,Winter,unknown
9748092,368,5104.0,2023-01-23 19:35:28,phone/Connect/NewDevice,0.000000,3.523319,3.0,0.0,0.0,21.5,...,0,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Winter,unknown


In [68]:
new_bluetooth_device = []

for session in tqdm(data_phone_os['session'].unique()):
    df = data_phone_os[data_phone_os['session'] == session].reset_index()
    df['new_bluetooth_device_to_pair'] = 'no bluetooth device to pair'
    
    new_device_indices = df[df['Label'].str.contains('NewDevice', na=False)].index
    if not new_device_indices.empty:
        start_index = max(0, new_device_indices[0] - window)  # Calculate start index
        end_index = new_device_indices[-1]  # Calculate end index
        print(start_index, end_index)
        # Update the new_bluetooth_device_to_pair column for the specified range
        df.loc[start_index:end_index, 'new_bluetooth_device_to_pair'] = 'new bluetooth device available'
    
    new_bluetooth_device.append(df)

# Concatenate all dataframes in new_bluetooth_device list
data_new_bluetooth_device = pd.concat(new_bluetooth_device, axis=0)

data_new_bluetooth_device = data_new_bluetooth_device.drop(columns=['index', 'level_0']).sort_values(by=['session', 'session'])
data_new_bluetooth_device = data_new_bluetooth_device.reset_index(drop=True)
data_new_bluetooth_device.to_csv('../data/06_context_feat_engg/data_featue_engineering.csv')

 23%|██▎       | 394/1704 [00:05<00:16, 79.16it/s]

0 38


 37%|███▋      | 626/1704 [00:08<00:15, 69.56it/s]

9102 9202


 96%|█████████▋| 1644/1704 [00:22<00:00, 65.77it/s]

268 368


100%|██████████| 1704/1704 [00:23<00:00, 72.22it/s]


In [69]:
data_new_bluetooth_device

,session,datetime,Label,avg_irradiation,steering_speed,temperature_out,light_sensor_rear,light_sensor_front,temperature_in,KBI_speed,...,current_drive_mode,current_clima_mode,current_media_source,nav_guidance_status,proximity_to_parking_spot,phone_status,bluetooth_connected,season,phone_os,new_bluetooth_device_to_pair
0,7.0,2022-09-07 20:08:33,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,...,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown,no bluetooth device to pair
1,7.0,2022-09-07 20:08:34,NaN,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.00,...,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown,no bluetooth device to pair
2,7.0,2022-09-07 20:08:35,NaN,0.000000,0.000000,25.0,0.0,0.000000,0.0,0.00,...,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown,no bluetooth device to pair
3,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,27.0,0.00,...,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown,no bluetooth device to pair
4,7.0,2022-09-07 20:08:36,NaN,0.000000,0.000000,25.0,0.0,0.000000,27.0,0.00,...,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Autumn,unknown,no bluetooth device to pair
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10124879,5399.0,2023-03-31 17:09:13,NaN,53.755324,0.000000,13.5,17600.0,6126.000000,28.5,92.48,...,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring,unconnected,no bluetooth device to pair
10124880,5399.0,2023-03-31 17:09:14,NaN,50.000000,0.000000,13.5,17200.0,5920.781157,28.5,93.12,...,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring,unconnected,no bluetooth device to pair
10124881,5399.0,2023-03-31 17:09:15,NaN,56.226313,15.000000,13.5,18000.0,5699.908827,28.5,93.44,...,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring,unconnected,no bluetooth device to pair
10124882,5399.0,2023-03-31 17:09:16,NaN,60.000000,0.000000,13.5,17600.0,5437.218229,28.5,93.76,...,car/driveMode/0.0,unknown,media/selectedSource/unavailable,navi_inactive,no_parking_spot_closeby,unconnected,bluetooth_unconnected,Spring,unconnected,no bluetooth device to pair


In [ ]:
#Data verification

cleaned_list = [x for x in context_data.Label.unique().tolist() if not pd.isna(x)]
print('no of reccomendation items:', len(cleaned_list))
sorted(cleaned_list)

In [ ]:

context_data.dropna()[context_data.dropna()['Label'].str.contains('goTo')].dropna()

In [ ]:
context_data[context_data['session']==502][['datetime', 'Label', 'session']+ status_static_context_var].dropna()

In [70]:
#find time delta between interactions
# interaction_time_delta = []
# data_new_bluetooth_device['datetime'] = pd.to_datetime(data_new_bluetooth_device['datetime'])
# for session in tqdm(data_new_bluetooth_device['session'].unique()):
#     df = data_new_bluetooth_device[data_new_bluetooth_device['session'] == session].reset_index()
#     df['timedelta'] = 0
#     indices = df.index[df['Label'].notna()]
#     # timedeltas = df.loc[indices, 'datetime'].diff()
#     # print(timedeltas.values)
#     # df['timedelta'] = pd.NaT
#     # df.loc[indices[0:], 'timedelta'] = timedeltas.values
#     df.loc[indices, 'timedelta'] = df.loc[indices, 'datetime'].diff().dt.total_seconds() / 60
#     df['timedelta'] = df['timedelta'].fillna(0)
#     interaction_time_delta.append(df)
# data_interaction_time_delta = pd.concat(interaction_time_delta, axis=0)
# data_interaction_time_delta[data_interaction_time_delta['session']==16][['session', 'timedelta','Label']].dropna()
# data_interaction_time_delta.to_csv('../data/05_Interaction_Sequences/context_newfeat.csv', index=False)